# Proyecto Final del curso de Ingeniería de Datos 

Se propone crear un pipeline que extraiga datos de una API pública de forma constante combinándolos con información extraída de una base de datos y colocándolos en un Data Warehouse.

## Setup

### Instalación de librerias

In [28]:
# Instalacion de la libreria para interactuar con la base de datos, especificamente con Postgres
#%pip install sqlalchemy psycopg2-binary


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importación de librerias

In [29]:
# Libreria para interactuar con APIs
import requests

import pandas as pd

# Libreria para interactuar con la base de datos
import sqlalchemy as sa

## Conexion con la API

Extraccion de datos de la API de transporte de Buenos Aires

In [30]:
base_url = "https://apitransporte.buenosaires.gob.ar"

### Extracción de datos de posiciones de los bus

In [10]:
# Obtencion de la posición de los vehículos monitoreados actualizada cada 30 segundos. 
# Si no se pasan parámetros de entrada, retorna la posición actual de todos los vehículos monitoreados.

endpoint_busPositions = "colectivos/vehiclePositions?json=1&client_id=33987e066c34484585f7d1c725e12831&client_secret=B72247DC28B7432BBCa552ED24E894C5"

full_url_busPositions = f"{base_url}/{endpoint_busPositions}"

r_busPositions = requests.get(full_url_busPositions)

In [11]:
r_busPositions.status_code

200

In [12]:
json_busData = r_busPositions.json()
json_busData

{'_entity': [{'_alert': None,
   '_id': '1',
   '_is_deleted': False,
   '_trip_update': None,
   '_vehicle': {'_congestion_level': 0,
    '_current_status': 2,
    '_current_stop_sequence': 0,
    '_occupancy_status': 0,
    '_position': {'_bearing': 0,
     '_latitude': -34.70582,
     '_longitude': -58.67087,
     '_odometer': 58823,
     '_speed': 0,
     'extensionObject': None},
    '_stop_id': None,
    '_timestamp': 1706796224,
    '_trip': None,
    '_vehicle': {'_id': '1830',
     '_label': '2829-922',
     '_license_plate': None,
     'extensionObject': None},
    'extensionObject': None},
   'extensionObject': None},
  {'_alert': None,
   '_id': '2',
   '_is_deleted': False,
   '_trip_update': None,
   '_vehicle': {'_congestion_level': 0,
    '_current_status': 2,
    '_current_stop_sequence': 0,
    '_occupancy_status': 0,
    '_position': {'_bearing': 0,
     '_latitude': -34.70468,
     '_longitude': -58.67005,
     '_odometer': 97715,
     '_speed': 5,
     'extensionOb

In [13]:
type(json_busData)

dict

In [14]:
json_busData.keys()

dict_keys(['_entity', '_header', 'extensionObject'])

Para pasar a un dataframe la data


In [15]:
# Para pasar el json a una dataframe
df_busPositions = pd.json_normalize(json_busData['_entity'])
df_busPositions.head()

,_alert,_id,_is_deleted,_trip_update,extensionObject,_vehicle._congestion_level,_vehicle._current_status,_vehicle._current_stop_sequence,_vehicle._occupancy_status,_vehicle._position._bearing,...,_vehicle._vehicle._license_plate,_vehicle._vehicle.extensionObject,_vehicle.extensionObject,_vehicle._trip._direction_id,_vehicle._trip._route_id,_vehicle._trip._schedule_relationship,_vehicle._trip._start_date,_vehicle._trip._start_time,_vehicle._trip._trip_id,_vehicle._trip.extensionObject
0,None,1,False,None,None,0,2,0,0,0,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,2,False,None,None,0,2,0,0,0,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,3,False,None,None,0,2,0,0,0,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,4,False,None,None,0,2,15,0,0,...,None,None,None,0.0,1468,0.0,20240201,10:45:00,95408-1,NaN
4,None,5,False,None,None,0,2,14,0,0,...,None,None,None,0.0,1466,0.0,20240201,11:00:00,95300-1,NaN


### Extracción de datos del estado de las estaciones de las ecobicis

In [16]:
# Obtencion del número de bicicletas y anclajes disponibles en cada estación y disponibilidad de estación.

endpoint_ecobiciSS = "ecobici/gbfs/stationStatus?client_id=33987e066c34484585f7d1c725e12831&client_secret=B72247DC28B7432BBCa552ED24E894C5"

full_url_ecobiciSS = f"{base_url}/{endpoint_ecobiciSS}"

r_ecobiciSS = requests.get(full_url_ecobiciSS)

In [17]:
r_ecobiciSS.status_code

200

In [18]:
json_ecobiciSS = r_ecobiciSS.json()
json_ecobiciSS

{'last_updated': 1706796483,
 'ttl': 6,
 'data': {'stations': [{'station_id': '2',
    'num_bikes_available': 1,
    'num_bikes_available_types': {'mechanical': 1, 'ebike': 0},
    'num_bikes_disabled': 0,
    'num_docks_available': 39,
    'num_docks_disabled': 0,
    'last_reported': 1706796239,
    'is_charging_station': False,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': '3',
    'num_bikes_available': 14,
    'num_bikes_available_types': {'mechanical': 14, 'ebike': 0},
    'num_bikes_disabled': 1,
    'num_docks_available': 13,
    'num_docks_disabled': 0,
    'last_reported': 1706796298,
    'is_charging_station': False,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': '4',
    'num_bikes_available': 0,
    'num_bikes_available_types': {'mechanical': 0, 'ebike': 0},
    'num_bikes_disabled': 2,
    'num_doc

In [19]:
json_ecobiciSS.keys()

dict_keys(['last_updated', 'ttl', 'data'])

In [26]:
# Para pasar el json a una dataframe

data_ecobiciSS= json_ecobiciSS['data']['stations']
df_ecobiciSS = pd.DataFrame(data_ecobiciSS)

df_ecobiciSS.head()

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic
0,2,1,"{'mechanical': 1, 'ebike': 0}",0,39,0,1.706796e+09,False,IN_SERVICE,1,1,1,None
1,3,14,"{'mechanical': 14, 'ebike': 0}",1,13,0,1.706796e+09,False,IN_SERVICE,1,1,1,None
2,4,0,"{'mechanical': 0, 'ebike': 0}",2,18,0,1.706796e+09,False,IN_SERVICE,1,1,1,None
3,5,2,"{'mechanical': 2, 'ebike': 0}",2,38,0,1.706796e+09,False,IN_SERVICE,1,1,1,None
4,6,16,"{'mechanical': 16, 'ebike': 0}",1,3,0,1.706796e+09,False,IN_SERVICE,1,1,1,None


## Conexión con base de datos